In [ ]:
import concurrent.futures
import os
import sys
from functools import partial

sys.path.append('..')

import matplotlib.pyplot as plt
import numpy as np

from materials import get_materials_Cvec
from safe_module import distance
from themes import colors
from themes import symmetry_classes

In [ ]:
pressures_BUK = [0.1, 2,  5, 10, 20, 50, 80, 100]
pressures_GRM = [0.1, 5, 10, 15, 20, 50, 80, 100]

In [ ]:
c_vecs = get_materials_Cvec(material="Aminzadeh")
n_materials = len(c_vecs)
n_pressures_BUK = len(pressures_BUK)
n_pressures_GRM = len(pressures_GRM)

n_symmetry_classes = len(symmetry_classes)

num_workers = os.cpu_count()

betas_BUK = np.zeros((n_symmetry_classes, n_pressures_BUK))
betas_GRM = np.zeros((n_symmetry_classes, n_pressures_GRM))

c_vecs   = c_vecs * n_symmetry_classes
sigmas   = [sigma for sigma in symmetry_classes for _ in range(n_materials)]

distance_new = partial(distance, use_parallel_processing=False)
with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
    results = list(executor.map(distance_new, c_vecs, sigmas))
    
betas, thetas, sigmas, phis = list(zip(*results))    

for i in range(n_symmetry_classes):
    index = i * n_materials
    betas_BUK[i,:]  = betas[index : index+n_pressures_BUK]
    betas_GRM[i,:]  = betas[index+n_pressures_BUK : index+n_materials]

In [ ]:
plt.figure(figsize=(6,10))

for i, sym_class in enumerate(symmetry_classes):
    plt.scatter(pressures_BUK, betas_BUK[i], color=colors[sym_class], s=30, label=sym_class)
    plt.plot(pressures_BUK, betas_BUK[i], color=colors[sym_class])

plt.xscale('log')
plt.legend(loc=1, fontsize=13, title=r'${\Sigma}$ (BUK)', title_fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylim(0,16)
plt.grid()
plt.xlabel("Pressure (MPa)", fontsize=20)
plt.ylabel(r'$\beta_{\Sigma}$ (°)', fontsize=20)
#plt.savefig('BUK.png')

In [ ]:
plt.figure(figsize=(6,10))

for i, sym_class in enumerate(symmetry_classes):
    plt.scatter(pressures_GRM, betas_GRM[i], color=colors[sym_class], s=30, label=sym_class)
    plt.plot(pressures_GRM, betas_GRM[i], color=colors[sym_class],)

plt.xscale('log')
plt.legend(loc=1, fontsize=13, title=r'${\Sigma}$ (GRM)', title_fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylim(0,16)
plt.grid()
plt.xlabel("Pressure (MPa)", fontsize=20)
plt.ylabel(r'$\beta_{\Sigma}$ (°)', fontsize=20)
#plt.savefig('GRM.png')